In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import os
import warnings
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:.2f}'.format

warnings.filterwarnings("ignore")

url_hourly = "https://media.githubusercontent.com/media/ruanvirginio/masters/refs/heads/main/bases_tratadas/transformers_dataset.csv"
df_hourly = pd.read_csv(url_hourly,  sep=';', encoding='latin-1')

url_daily = "https://media.githubusercontent.com/media/ruanvirginio/masters/refs/heads/main/bases_tratadas/daily_peak_transformers_dataset.csv"
df_daily = pd.read_csv(url_daily,  sep=';', encoding='latin-1')

# df_count = df_daily.groupby('id').count()

# df_count = df_count.sort_values('datahora').tail(41).reset_index() # esses são os trafos com mais de 97,5% de linhas preenchidas
# trafos_escolhidos = df_count['id'].unique().tolist()
# df_filtrado = df_daily[df_daily['id'].isin(trafos_escolhidos)]

# fig_aparente = px.line(df_filtrado, x='datahora', y='S', color='id',
#                        title='Potência Aparente ao Longo do Tempo por Transformador',
#                        labels={'S': 'Potência Aparente (kVA)', 'Dia': 'Data'})

# fig_aparente.show()
# fig_aparente.write_html("Demanda ao longo do tempo - IQR.html")


#### Algoritmo de Aprendizado de Máquina - Pico Diário, 1 feature

In [ ]:
# ========================================================================
# IMPORTAÇÕES E CONFIGURAÇÕES
# ========================================================================
import os
import time
import random
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'browser'

from math import sqrt
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten
import joblib

import matplotlib
matplotlib.use('Agg')

# Fixando seeds
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

# ========================================================================
# FUNÇÕES AUXILIARES
# ========================================================================
def gerar_tabela_metricas_por_fold(trafo, modelo, fold_rmse, fold_mae):
    return pd.DataFrame({
        'Fold': [f'Fold {i+1}' for i in range(len(fold_rmse))],
        'Trafo': trafo,
        'Modelo': modelo,
        'RMSE': np.round(fold_rmse, 4),
        'MAE': np.round(fold_mae, 4)
    })


def plotar_resultados(datas, y_real, y_pred, trafo, modelo):
    plt.figure(figsize=(14, 6))
    plt.plot(datas, y_real, label='Real', color='blue')
    plt.plot(datas, y_pred, label=f'Previsto ({modelo})', linestyle='--', color='orange')
    plt.xlabel('Data', fontsize=16)
    plt.ylabel('Potência Aparente (kVA)', fontsize=16)
    plt.title(f'Previsão - {modelo} ({trafo})', fontsize=18)
    plt.legend(fontsize=14)
    plt.tight_layout()

    os.makedirs('plots', exist_ok=True)
    plt.savefig(f'plots/PLOT_{modelo}_{trafo}.pdf', format='pdf', dpi=300, bbox_inches='tight')
    plt.show()


def plotar_todos_folds(lista_datas, lista_reais, lista_previstos, trafo, modelo):
    datas_todas = pd.to_datetime(np.concatenate(lista_datas))
    reais_todos = np.concatenate(lista_reais)
    previstos_todos = np.concatenate(lista_previstos)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=datas_todas, y=reais_todos, mode='lines', name='Real', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=datas_todas, y=previstos_todos, mode='lines', name=f'Previsto ({modelo})', line=dict(color='orange', dash='dash')))

    fig.update_layout(
        title=f'Previsão em Todos os Folds - {trafo} ({modelo})',
        xaxis_title='Data',
        yaxis_title='Potência Aparente',
        hovermode='x unified'
    )
    fig.show()

# ========================================================================
# FUNÇÃO PRINCIPAL DE TREINAMENTO E PREVISÃO
# ========================================================================
def treinar_e_prever_modelo(data, trafos_escolhidos, modelo, janela, epochs=20, batch_size=32):
    resultados = []
    os.makedirs('modelos', exist_ok=True)

    for trafo in trafos_escolhidos:
        print(f"\n🔹 Treinando {modelo} para {trafo}...")

        df = data[data['id'] == trafo].copy()
        df = df[['datahora', 'S']].set_index('datahora').sort_index()

        # Cria janelas deslizantes
        X, y = [], []
        for i in range(janela, len(df)):
            X.append(df.iloc[i-janela:i].values)
            y.append(df.iloc[i, 0])
        X, y = np.array(X), np.array(y)

        # Ajuste de shape para redes neurais
        if modelo in ['LSTM', 'CNN', 'CNN_LSTM']:
            X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))

        tscv = TimeSeriesSplit(n_splits=5)
        fold_rmse, fold_mae = [], []
        lista_datas, lista_reais, lista_previstos = [], [], []

        for fold_idx, (train_idx, test_idx) in enumerate(tscv.split(X)):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            # Inicializa modelo
            if modelo == 'SVR':
                regressor = SVR(kernel='rbf', C=100, gamma=0.001, epsilon=0.01)
            elif modelo == 'RFR':
                regressor = RandomForestRegressor(n_estimators=100, max_depth=20, random_state=42, n_jobs=-1)
            elif modelo == 'GBR':
                regressor = GradientBoostingRegressor(n_estimators=100, random_state=42)
            elif modelo == 'LGBM':
                regressor = LGBMRegressor(n_estimators=100, random_state=42)
            elif modelo == 'XGB':
                regressor = XGBRegressor(n_estimators=100, random_state=42)
            elif modelo == 'LSTM':
                regressor = Sequential([
                    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
                    LSTM(50),
                    Dense(1)
                ])
                regressor.compile(optimizer='adam', loss='mse')
            elif modelo == 'CNN':
                regressor = Sequential([
                    Conv1D(64, 2, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
                    MaxPooling1D(2),
                    Flatten(),
                    Dense(50, activation='relu'),
                    Dense(1)
                ])
                regressor.compile(optimizer='adam', loss='mse')
            elif modelo == 'CNN_LSTM':
                regressor = Sequential([
                    Conv1D(64, 2, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
                    MaxPooling1D(2),
                    LSTM(50, return_sequences=True),
                    LSTM(50),
                    Dense(1)
                ])
                regressor.compile(optimizer='adam', loss='mse')

            # Treinamento
            if modelo in ['LSTM', 'CNN', 'CNN_LSTM']:
                regressor.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
                y_pred = regressor.predict(X_test)
            else:
                regressor.fit(X_train.reshape(X_train.shape[0], -1), y_train)
                y_pred = regressor.predict(X_test.reshape(X_test.shape[0], -1))

            # Métricas
            rmse = sqrt(mean_squared_error(y_test, y_pred))
            mae = mean_absolute_error(y_test, y_pred)
            fold_rmse.append(rmse)
            fold_mae.append(mae)

            datas_finais = df.index[test_idx]
            lista_datas.append(datas_finais)
            lista_reais.append(y_test)
            lista_previstos.append(y_pred)

            if fold_idx == 4:
                plotar_resultados(datas_finais, y_test, y_pred, trafo, modelo)

        # Salva modelo final
        modelo_path = f"modelos/{modelo}_{trafo}.{'h5' if modelo in ['LSTM', 'CNN', 'CNN_LSTM'] else 'pkl'}"
        if modelo in ['LSTM', 'CNN', 'CNN_LSTM']:
            regressor.save(modelo_path)
        else:
            joblib.dump(regressor, modelo_path)
        print(f"✅ Modelo salvo em: {modelo_path}")

        df_metricas = gerar_tabela_metricas_por_fold(trafo, modelo, fold_rmse, fold_mae)
        print(df_metricas)

        plotar_todos_folds(lista_datas, lista_reais, lista_previstos, trafo, modelo)

        resultados.append({
            'Trafo': trafo,
            'Modelo': modelo,
            'RMSE Médio': np.round(np.mean(fold_rmse), 4),
            'MAE Médio': np.round(np.mean(fold_mae), 4)
        })

    return pd.DataFrame(resultados)


# ========================================================================
# df_daily deve estar carregado e normalizado
trafos = ['T1', 'T2'] 
# resultados_SVR = treinar_e_prever_modelo(df_daily, trafos, modelo='SVR', janela=365)
# resultados_LSTM = treinar_e_prever_modelo(df_daily, trafos, modelo='LSTM', janela=365)
# resultados_CNN = treinar_e_prever_modelo(df_daily, trafos, modelo='CNN', janela=365)
# resultados_RFR = treinar_e_prever_modelo(df_daily, trafos, modelo='RFR', janela=365)
resultados_GBR = treinar_e_prever_modelo(df_daily, trafos, modelo='GBR', janela=365)
resultados_XBR = treinar_e_prever_modelo(df_daily, trafos, modelo='XBR', janela=365)
resultados_LGBM = treinar_e_prever_modelo(df_daily, trafos, modelo='LGBM', janela=365)
resultados_CNN_LSTM = treinar_e_prever_modelo(df_daily, trafos, modelo='CNN_LSTM', janela=365)


In [ ]:
import os
from tensorflow.keras.models import load_model
import joblib

# Lista de transformadores e modelos
trafos = ['T1', 'T2']
modelos = ['SVR', 'LSTM', 'CNN', 'CNN_LSTM']

# Diretório base onde estão os modelos
base_dir = r"G:\Meu Drive\Estudos\Mestrado\Github\masters\modelos"

# Dicionário para armazenar os modelos carregados
modelos_carregados = {}

for trafo in trafos:
    for modelo in modelos:
        # Define a extensão correta
        extensao = "h5" if modelo in ["LSTM", "CNN", "CNN_LSTM"] else "pkl"
        caminho = os.path.join(base_dir, f"{modelo}_{trafo}.{extensao}")
        
        # Inicializa a chave no dicionário
        modelos_carregados[(modelo, trafo)] = None
        
        # Carrega apenas se o arquivo existir
        if os.path.exists(caminho):
            if modelo in ["LSTM", "CNN", "CNN_LSTM"]:
                modelos_carregados[(modelo, trafo)] = load_model(caminho)
            else:
                modelos_carregados[(modelo, trafo)] = joblib.load(caminho)
            print(f"✅ Modelo carregado: {caminho}")
        else:
            print(f"⚠️ Arquivo não encontrado, ignorando: {caminho}")


In [ ]:
# Vamos analisar seus dados originais para entender a estrutura
print("=== ANALISANDO DADOS ORIGINAIS ===")
print(f"Shape do df_daily: {df_daily.shape}")
print(f"Colunas: {df_daily.columns.tolist()}")
print(f"Tipos de dados:\n{df_daily.dtypes}")

# Ver um exemplo dos dados do T1
print("\n=== DADOS DO TRAFO T1 ===")
df_t1 = df_daily[df_daily['id'] == 'T1'].copy()
print(f"Quantidade de dados T1: {len(df_t1)}")
print(f"Período: {df_t1['datahora'].min()} até {df_t1['datahora'].max()}")
print(f"Primeiras linhas T1:")
print(df_t1.head())

# Ver a distribuição da variável target 'S'
print(f"\nEstatísticas da Potência Aparente (S):")
print(df_t1['S'].describe())

### Diário, multi-feature

In [ ]:
def criar_features_daily(df_daily):
    df_featured_daily = df_daily.copy()
        
    # Features para dados diários
    df_featured_daily['datahora'] = pd.to_datetime(df_featured_daily['datahora'])
    df_featured_daily = df_featured_daily.sort_values(['id', 'datahora'])
    
    df_featured_daily['day_of_week'] = df_featured_daily['datahora'].dt.dayofweek
    df_featured_daily['month'] = df_featured_daily['datahora'].dt.month
    df_featured_daily['year'] = df_featured_daily['datahora'].dt.year
    df_featured_daily['is_weekend'] = (df_featured_daily['day_of_week'] >= 5).astype(int)
    
    # Features cíclicas diárias
    df_featured_daily['sin_day_of_week'] = np.sin(2 * np.pi * df_featured_daily['day_of_week'] / 7)
    df_featured_daily['cos_day_of_week'] = np.cos(2 * np.pi * df_featured_daily['day_of_week'] / 7)
    df_featured_daily['sin_month'] = np.sin(2 * np.pi * df_featured_daily['month'] / 12)
    df_featured_daily['cos_month'] = np.cos(2 * np.pi * df_featured_daily['month'] / 12)
    
    # Lags diários (sazonais)
    for lag in [1, 7, 30, 365]:  # 1 dia, 1 semana, 1 mês, 1 ano
        df_featured_daily[f'S{lag}'] = df_featured_daily.groupby('id')['S'].shift(lag)
    
    # Médias móveis diárias
    for window in [7, 30, 90]:  # 1 semana, 1 mês, 3 meses
        df_featured_daily[f'S_rolling_mean_{window}'] = df_featured_daily.groupby('id')['S'].transform(
            lambda x: x.rolling(window, min_periods=1).mean())
        
    return df_featured_daily

print("\nCriando features de engenharia de tempo para dados diários...")
daily_features = criar_features_daily(df_daily)
daily_features
# data_com_features.dropna(inplace=True)
print("✅ Features diárias criadas com sucesso.")


### Hora em Hora, multi-feature

In [ ]:
def criar_features_hourly(df_hourly):
    # """Cria features específicas para cada modo"""    
    df_featured_hourly = df_hourly.copy()

    # Features para dados horários
    df_featured_hourly['datahora'] = pd.to_datetime(df_featured_hourly['datahora'])
    df_featured_hourly = df_featured_hourly.sort_values(['id', 'datahora'])
    
    df_featured_hourly['hour'] = df_featured_hourly['datahora'].dt.hour
    df_featured_hourly['day_of_week'] = df_featured_hourly['datahora'].dt.dayofweek
    df_featured_hourly['is_weekend'] = (df_featured_hourly['day_of_week'] >= 5).astype(int)
    
    # Features cíclicas horárias
    df_featured_hourly['sin_hour'] = np.sin(2 * np.pi * df_featured_hourly['hour'] / 24)
    df_featured_hourly['cos_hour'] = np.cos(2 * np.pi * df_featured_hourly['hour'] / 24)
    
    # Lags horários
    for lag in [1, 24, 168]:  # 1h, 1 dia, 1 semana
        df_featured_hourly[f'S_lag_{lag}'] = df_featured_hourly.groupby('id')['S'].shift(lag)
    
    # Médias móveis horárias
    df_featured_hourly['S_rolling_mean_24'] = df_featured_hourly.groupby('id')['S'].transform(
        lambda x: x.rolling(24, min_periods=1).mean())
        
        
    return df_featured_hourly

print("\nCriando features de engenharia de tempo para dados horários...")
hourly_features = criar_features_hourly(df_hourly)

# data_com_features.dropna(inplace=True)
print("✅ Features horárias criadas com sucesso.")
